<a href="https://colab.research.google.com/github/Ph1lipXu/Machine-Learning-on-Suicide-and-Depression-Detection/blob/main/Modeling_16000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
#!pip install tensorflow nltk scikit-learn
#!pip install gensim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, SimpleRNN, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import gensim
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision = 4, suppress = True)

In [3]:
!wget https://raw.githubusercontent.com/Ph1lipXu/Machine-Learning-on-Suicide-and-Depression-Detection/refs/heads/main/data/cleaned_data_16000.csv

--2025-04-10 19:50:01--  https://raw.githubusercontent.com/Ph1lipXu/Machine-Learning-on-Suicide-and-Depression-Detection/refs/heads/main/data/cleaned_data_16000.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36085156 (34M) [text/plain]
Saving to: ‘cleaned_data_16000.csv’

cleaned_data_16000. 100%[===================>]  34.41M   112MB/s    in 0.3s    

2025-04-10 19:50:03 (112 MB/s) - ‘cleaned_data_16000.csv’ saved [36085156/36085156]



In [4]:
df = pd.read_csv("/content/cleaned_data_16000.csv")
df.head()

,text,class,tokens
0,can i get some support please...so i am not as depressed as i used to be (i ...,depression,"['can', 'I', 'get', 'some', 'support', 'please', 'so', 'I', 'be', 'not', 'as..."
1,"everything is going wrong .i have been trying not to drink, but everyone is ...",depression,"['everything', 'be', 'go', 'wrong', 'have', 'be', 'try', 'not', 'to', 'drink..."
2,i am done fighting it.*gone*,depression,"['I', 'be', 'do', 'fight', 'it', 'go']"
3,today i cut my hairmy hair has always been a thick mess of curls that went a...,depression,"['today', 'I', 'cut', 'my', 'hairmy', 'hair', 'have', 'always', 'be', 'a', '..."
4,i do not know what to do and i have no hopes for the future.it is kinda toug...,depression,"['I', 'do', 'not', 'know', 'what', 'to', 'do', 'and', 'I', 'have', 'no', 'ho..."


# Modeling

In [6]:
df['suicide_class'] = df['class'].apply(lambda x: 'suicide' if x == 'SuicideWatch' else 'nonsuicide')
df['depression_class'] = df['class'].apply(lambda x: 'depression' if x == 'depression' else 'nondepression')
df['teenager_class'] = df['class'].apply(lambda x: 'teenager' if x == 'teenagers' else 'nonteenager')

In [8]:
df.head(10)

,text,class,tokens,suicide_class,depression_class,teenager_class
0,can i get some support please...so i am not as depressed as i used to be (i ...,depression,"['can', 'I', 'get', 'some', 'support', 'please', 'so', 'I', 'be', 'not', 'as...",nonsuicide,depression,nonteenager
1,"everything is going wrong .i have been trying not to drink, but everyone is ...",depression,"['everything', 'be', 'go', 'wrong', 'have', 'be', 'try', 'not', 'to', 'drink...",nonsuicide,depression,nonteenager
2,i am done fighting it.*gone*,depression,"['I', 'be', 'do', 'fight', 'it', 'go']",nonsuicide,depression,nonteenager
3,today i cut my hairmy hair has always been a thick mess of curls that went a...,depression,"['today', 'I', 'cut', 'my', 'hairmy', 'hair', 'have', 'always', 'be', 'a', '...",nonsuicide,depression,nonteenager
4,i do not know what to do and i have no hopes for the future.it is kinda toug...,depression,"['I', 'do', 'not', 'know', 'what', 'to', 'do', 'and', 'I', 'have', 'no', 'ho...",nonsuicide,depression,nonteenager
5,"tired of life, tired of living. do not know what to do hey guys, \n\ni am 16...",depression,"['tired', 'of', 'life', 'tired', 'of', 'live', 'do', 'not', 'know', 'what', ...",nonsuicide,depression,nonteenager
6,what is one concrete thing that has helped you in your battle against depres...,depression,"['what', 'be', 'one', 'concrete', 'thing', 'that', 'have', 'help', 'you', 'i...",nonsuicide,depression,nonteenager
7,does mental health go hand in hand with the physical health?when i feel at m...,depression,"['do', 'mental', 'health', 'go', 'hand', 'in', 'hand', 'with', 'the', 'physi...",nonsuicide,depression,nonteenager
8,the thing that hurts the most is knowing that i have been through worse.when...,depression,"['the', 'thing', 'that', 'hurt', 'the', 'most', 'be', 'know', 'that', 'I', '...",nonsuicide,depression,nonteenager
9,need someone to talk toi am a guy in high school and i just need to talk to ...,depression,"['need', 'someone', 'to', 'talk', 'toi', 'be', 'a', 'guy', 'in', 'high', 'sc...",nonsuicide,depression,nonteenager


## Suicide / Non-Suicide

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
processed_df["label"] = label_encoder.fit_transform(processed_df["suicide_class"])
num_classes = len(label_encoder.classes_)

In [ ]:
from sklearn.model_selection import train_test_split

# Instead of random train test split, stratify by class
train_texts, test_texts, train_labels, test_labels = train_test_split(
    processed_df["tokens"], processed_df["label"], random_state=64, stratify=processed_df['label']
)
print('Training data: ',len(train_texts))
print('Testing data: ',len(test_texts))

Training data:  12000
Testing data:  4000


In [ ]:
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [ ]:
print(train_texts.head(10))

1435     [anyone, else, feel, like, this, I, have, be, cope, with, depression, for, a...
1368     [lose, my, sense, of, realityit, be, another, one, of, thoughse, night, and,...
5863                                                    [good, way, to, commit, suicide]
8929     [I, be, bakk, you, lousy, son, of, bitch, thoughught, I, be, do, I, give, yo...
15448                                                                [no, text, content]
11663    [just, realize, how, close, my, teenage, year, be, to, finish, I, be, and, I...
3897     [just, a, thoughught, that, enter, my, mind, lie, in, bed, at, not, able, to...
4635     [why, when, I, die, the, world, will, not, stop, spin, nothing, change, ti, ...
14158                                                                [no, text, content]
13454                                                                [no, text, content]
Name: tokens, dtype: object


### Vectorization/Embedding

In [ ]:
# Initialize tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding sequences to have the same length
max_len = 200  # Max length for padding
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Vocabulary size
vocab_size = len(word_index) + 1

In [ ]:
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

In [ ]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score
class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_data, val_labels = self.validation_data
        val_preds = self.model.predict(val_data)
        val_preds = np.argmax(val_preds, axis=1)  # Convert probabilities to class labels
        val_labels = np.argmax(val_labels, axis=1)  # Convert one-hot labels to class labels

        f1 = f1_score(val_labels, val_preds, average='weighted')  # Change to 'macro' if needed
        print(f" - val_f1: {f1:.4f}")
        logs["val_f1"] = f1  # Store it in logs if needed

In [ ]:
def build_cnn_model(vocab_size, embedding_matrix, num_classes):
    model = Sequential([
        Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_rnn_model(vocab_size, embedding_matrix, num_classes):
    model = Sequential([
        Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False),
        LSTM(128, return_sequences=False),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_bilstm_model(vocab_size, embedding_matrix, num_classes):
    model = Sequential([
        Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False),
        Bidirectional(LSTM(128)),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def train_and_evaluate(model, train_padded, train_labels, test_padded, test_labels, epochs=10, batch_size=32):
    f1_callback = F1ScoreCallback(validation_data=(test_padded, test_labels))

    model.fit(train_padded, train_labels,
              epochs=epochs, batch_size=batch_size,
              validation_data=(test_padded, test_labels),
              callbacks=[f1_callback])

    loss, acc = model.evaluate(test_padded, test_labels)
    print(f"Test Accuracy: {acc:.4f}")

In [ ]:
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 184ms/step
 - val_f1: 0.8046
375/375 ━━━━━━━━━━━━━━━━━━━━ 253s 660ms/step - accuracy: 0.7535 - loss: 0.4779 - val_accuracy: 0.8127 - val_loss: 0.3835 - val_f1: 0.8046
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 182ms/step
 - val_f1: 0.8145
375/375 ━━━━━━━━━━━━━━━━━━━━ 247s 660ms/step - accuracy: 0.8110 - loss: 0.3813 - val_accuracy: 0.8242 - val_loss: 0.3606 - val_f1: 0.8145
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 185ms/step
 - val_f1: 0.7994
375/375 ━━━━━━━━━━━━━━━━━━━━ 287s 726ms/step - accuracy: 0.8303 - loss: 0.3551 - val_accuracy: 0.8150 - val_loss: 0.4182 - val_f1: 0.7994
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 184ms/step
 - val_f1: 0.8228
375/375 ━━━━━━━━━━━━━━━━━━━━ 234s 623ms/step - accuracy: 0.8340 - loss: 0.3535 - val_accuracy: 0.8330 - val_loss: 0.3648 - val_f1: 0.8228
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 172ms/step
 - val_f1: 0.8388
375/375 ━━━━━━━━━━━━━━━━━━━━ 229s 611ms/step - accuracy: 0.8643 - loss: 0.3010 - va

In [ ]:
print("Training Bi-LSTM with FastText Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 24s 185ms/step
 - val_f1: 0.7444
375/375 ━━━━━━━━━━━━━━━━━━━━ 259s 681ms/step - accuracy: 0.7477 - loss: 0.4818 - val_accuracy: 0.7845 - val_loss: 0.3952 - val_f1: 0.7444
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 186ms/step
 - val_f1: 0.8111
375/375 ━━━━━━━━━━━━━━━━━━━━ 277s 722ms/step - accuracy: 0.7967 - loss: 0.4043 - val_accuracy: 0.8045 - val_loss: 0.3845 - val_f1: 0.8111
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 24s 191ms/step
 - val_f1: 0.7504
375/375 ━━━━━━━━━━━━━━━━━━━━ 324s 729ms/step - accuracy: 0.8294 - loss: 0.3570 - val_accuracy: 0.7383 - val_loss: 0.4499 - val_f1: 0.7504
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 186ms/step
 - val_f1: 0.8132
375/375 ━━━━━━━━━━━━━━━━━━━━ 303s 678ms/step - accuracy: 0.8182 - loss: 0.3601 - val_accuracy: 0.8090 - val_loss: 0.3803 - val_f1: 0.8132
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 184ms/step
 - val_f1: 0.8259
375/375 ━━━━━━━━━━━━━━━━━━━━ 264s 706ms/step - accuracy: 0.8643 - loss: 0.2955 - val_accuracy:

In [ ]:
print("Training CNN with Word2Vec Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
 - val_f1: 0.7749
375/375 ━━━━━━━━━━━━━━━━━━━━ 25s 62ms/step - accuracy: 0.7245 - loss: 0.6417 - val_accuracy: 0.8025 - val_loss: 0.3911 - val_f1: 0.7749
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step
 - val_f1: 0.8299
375/375 ━━━━━━━━━━━━━━━━━━━━ 23s 60ms/step - accuracy: 0.8186 - loss: 0.3763 - val_accuracy: 0.8307 - val_loss: 0.3693 - val_f1: 0.8299
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.8365
375/375 ━━━━━━━━━━━━━━━━━━━━ 22s 58ms/step - accuracy: 0.8435 - loss: 0.3392 - val_accuracy: 0.8388 - val_loss: 0.3523 - val_f1: 0.8365
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
 - val_f1: 0.8305
375/375 ━━━━━━━━━━━━━━━━━━━━ 42s 60ms/step - accuracy: 0.8601 - loss: 0.3041 - val_accuracy: 0.8245 - val_loss: 0.3658 - val_f1: 0.8305
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
 - val_f1: 0.8359
375/375 ━━━━━━━━━━━━━━━━━━━━ 22s 58ms/step - accuracy: 0.8777 - loss: 0.2698 - val_accuracy: 0.8317 - val_loss: 

In [ ]:
print("Training CNN with FastText Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.6463
375/375 ━━━━━━━━━━━━━━━━━━━━ 25s 62ms/step - accuracy: 0.7104 - loss: 0.6730 - val_accuracy: 0.7513 - val_loss: 0.4629 - val_f1: 0.6463
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.7725
375/375 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - accuracy: 0.7590 - loss: 0.4579 - val_accuracy: 0.7990 - val_loss: 0.4166 - val_f1: 0.7725
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step
 - val_f1: 0.7673
375/375 ━━━━━━━━━━━━━━━━━━━━ 43s 65ms/step - accuracy: 0.7898 - loss: 0.4229 - val_accuracy: 0.8000 - val_loss: 0.4021 - val_f1: 0.7673
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.8186
375/375 ━━━━━━━━━━━━━━━━━━━━ 22s 58ms/step - accuracy: 0.8060 - loss: 0.4014 - val_accuracy: 0.8238 - val_loss: 0.3831 - val_f1: 0.8186
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.8108
375/375 ━━━━━━━━━━━━━━━━━━━━ 42s 61ms/step - accuracy: 0.8185 - loss: 0.3749 - val_accuracy: 0.8213 - val_loss: 

### Tokenized


In [ ]:
label_encoder = LabelEncoder()
processed_df["label"] = label_encoder.fit_transform(processed_df["suicide_class"])
num_classes = len(label_encoder.classes_)

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(processed_df["tokens"], processed_df["label"], test_size=0.2, random_state=64)

# Train Word2Vec and FastText models
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts.apply(' '.join))  # Join tokens back to text for the tokenizer
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Create embedding matrices
embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

max_len = 100  # Max length for padding

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts.apply(' '.join))  # Join tokens for sequences
test_sequences = tokenizer.texts_to_sequences(test_texts.apply(' '.join))

# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Convert labels to categorical
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Train Bi-LSTM with Word2Vec embeddings
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 96ms/step
 - val_f1: 0.7907
400/400 ━━━━━━━━━━━━━━━━━━━━ 127s 302ms/step - accuracy: 0.7560 - loss: 0.4691 - val_accuracy: 0.8072 - val_loss: 0.3875 - val_f1: 0.7907
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 97ms/step
 - val_f1: 0.8307
400/400 ━━━━━━━━━━━━━━━━━━━━ 119s 297ms/step - accuracy: 0.8197 - loss: 0.3654 - val_accuracy: 0.8328 - val_loss: 0.3527 - val_f1: 0.8307
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 96ms/step
 - val_f1: 0.8270
400/400 ━━━━━━━━━━━━━━━━━━━━ 142s 297ms/step - accuracy: 0.8354 - loss: 0.3416 - val_accuracy: 0.8309 - val_loss: 0.3565 - val_f1: 0.8270
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step
 - val_f1: 0.7921
400/400 ━━━━━━━━━━━━━━━━━━━━ 166s 356ms/step - accuracy: 0.8559 - loss: 0.3130 - val_accuracy: 0.8119 - val_loss: 0.3930 - val_f1: 0.7921
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 95ms/step
 - val_f1: 0.8242
400/400 ━━━━━━━━━━━━━━━━━━━━ 181s 304ms/step - accuracy: 0.8733 - loss: 0.2758 - val_accuracy: 0.8

## Depression / Non-Depression

In [ ]:
depression_df['depression_class'] = depression_df['class'].apply(lambda x: 'depression' if x == 'depression' else 'nondepression')

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
depression_df["label"] = label_encoder.fit_transform(depression_df["depression_class"])
num_classes = len(label_encoder.classes_)

In [ ]:
from sklearn.model_selection import train_test_split

# Instead of random train test split, stratify by class
train_texts, test_texts, train_labels, test_labels = train_test_split(
    depression_df["tokens"], depression_df["label"], random_state=64, stratify=depression_df['label']
)

In [ ]:
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

### Vectorization/Embedding

In [ ]:
# Initialize tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding sequences to have the same length
max_len = 200  # Max length for padding
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Vocabulary size
vocab_size = len(word_index) + 1

In [ ]:
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

In [ ]:
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 182ms/step
 - val_f1: 0.7729
375/375 ━━━━━━━━━━━━━━━━━━━━ 234s 611ms/step - accuracy: 0.7374 - loss: 0.4883 - val_accuracy: 0.7828 - val_loss: 0.4120 - val_f1: 0.7729
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 183ms/step
 - val_f1: 0.7709
375/375 ━━━━━━━━━━━━━━━━━━━━ 296s 705ms/step - accuracy: 0.8076 - loss: 0.3888 - val_accuracy: 0.7815 - val_loss: 0.4130 - val_f1: 0.7709
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 176ms/step
 - val_f1: 0.8040
375/375 ━━━━━━━━━━━━━━━━━━━━ 325s 712ms/step - accuracy: 0.8210 - loss: 0.3690 - val_accuracy: 0.8008 - val_loss: 0.4021 - val_f1: 0.8040
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 179ms/step
 - val_f1: 0.7947
375/375 ━━━━━━━━━━━━━━━━━━━━ 290s 623ms/step - accuracy: 0.8395 - loss: 0.3433 - val_accuracy: 0.8130 - val_loss: 0.3881 - val_f1: 0.7947
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 183ms/step
 - val_f1: 0.8212
375/375 ━━━━━━━━━━━━━━━━━━━━ 254s 605ms/step - accuracy: 0.8505 - loss: 0.3219 - val_accuracy:

In [ ]:
print("Training Bi-LSTM with FastText Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 183ms/step
 - val_f1: 0.7544
375/375 ━━━━━━━━━━━━━━━━━━━━ 234s 611ms/step - accuracy: 0.7471 - loss: 0.4862 - val_accuracy: 0.7720 - val_loss: 0.4271 - val_f1: 0.7544
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 181ms/step
 - val_f1: 0.7745
375/375 ━━━━━━━━━━━━━━━━━━━━ 283s 667ms/step - accuracy: 0.7752 - loss: 0.4164 - val_accuracy: 0.7720 - val_loss: 0.4150 - val_f1: 0.7745
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 186ms/step
 - val_f1: 0.7767
375/375 ━━━━━━━━━━━━━━━━━━━━ 263s 669ms/step - accuracy: 0.7956 - loss: 0.3904 - val_accuracy: 0.7872 - val_loss: 0.4048 - val_f1: 0.7767
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 173ms/step
 - val_f1: 0.7773
375/375 ━━━━━━━━━━━━━━━━━━━━ 268s 685ms/step - accuracy: 0.8163 - loss: 0.3710 - val_accuracy: 0.7945 - val_loss: 0.4036 - val_f1: 0.7773
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 23s 183ms/step
 - val_f1: 0.7789
375/375 ━━━━━━━━━━━━━━━━━━━━ 236s 615ms/step - accuracy: 0.8379 - loss: 0.3372 - val_accuracy:

In [ ]:
print("Training CNN with Word2Vec Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.7455
375/375 ━━━━━━━━━━━━━━━━━━━━ 25s 62ms/step - accuracy: 0.7287 - loss: 0.5791 - val_accuracy: 0.7897 - val_loss: 0.4091 - val_f1: 0.7455
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.8069
375/375 ━━━━━━━━━━━━━━━━━━━━ 39s 58ms/step - accuracy: 0.7999 - loss: 0.4054 - val_accuracy: 0.8160 - val_loss: 0.3857 - val_f1: 0.8069
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step
 - val_f1: 0.8186
375/375 ━━━━━━━━━━━━━━━━━━━━ 44s 66ms/step - accuracy: 0.8152 - loss: 0.3702 - val_accuracy: 0.8195 - val_loss: 0.3854 - val_f1: 0.8186
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.8092
375/375 ━━━━━━━━━━━━━━━━━━━━ 40s 62ms/step - accuracy: 0.8377 - loss: 0.3393 - val_accuracy: 0.8195 - val_loss: 0.3797 - val_f1: 0.8092
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
 - val_f1: 0.8204
375/375 ━━━━━━━━━━━━━━━━━━━━ 28s 74ms/step - accuracy: 0.8523 - loss: 0.3076 - val_accuracy: 0.8163 - val_loss: 

In [ ]:
print("Training CNN with FastText Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step
 - val_f1: 0.6427
375/375 ━━━━━━━━━━━━━━━━━━━━ 25s 62ms/step - accuracy: 0.7141 - loss: 0.6985 - val_accuracy: 0.7498 - val_loss: 0.4588 - val_f1: 0.6427
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.6440
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - accuracy: 0.7494 - loss: 0.4668 - val_accuracy: 0.7505 - val_loss: 0.4421 - val_f1: 0.6440
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.7338
375/375 ━━━━━━━━━━━━━━━━━━━━ 23s 61ms/step - accuracy: 0.7609 - loss: 0.4383 - val_accuracy: 0.7695 - val_loss: 0.4333 - val_f1: 0.7338
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
 - val_f1: 0.6432
375/375 ━━━━━━━━━━━━━━━━━━━━ 40s 58ms/step - accuracy: 0.7592 - loss: 0.4231 - val_accuracy: 0.7498 - val_loss: 0.4468 - val_f1: 0.6432
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.6981
375/375 ━━━━━━━━━━━━━━━━━━━━ 43s 64ms/step - accuracy: 0.7562 - loss: 0.4272 - val_accuracy: 0.7638 - val_loss: 

### Tokenized

In [ ]:
label_encoder = LabelEncoder()
depression_df["label"] = label_encoder.fit_transform(processed_df["depression_class"])
num_classes = len(label_encoder.classes_)

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(depression_df["tokens"], depression_df["label"], test_size=0.2, random_state=64)

# Train Word2Vec and FastText models
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts.apply(' '.join))  # Join tokens back to text for the tokenizer
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Create embedding matrices
embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

max_len = 100  # Max length for padding

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts.apply(' '.join))  # Join tokens for sequences
test_sequences = tokenizer.texts_to_sequences(test_texts.apply(' '.join))

# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Convert labels to categorical
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Train Bi-LSTM with Word2Vec embeddings
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 84ms/step
 - val_f1: 0.7724
400/400 ━━━━━━━━━━━━━━━━━━━━ 133s 314ms/step - accuracy: 0.7491 - loss: 0.4769 - val_accuracy: 0.7653 - val_loss: 0.4281 - val_f1: 0.7724
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step
 - val_f1: 0.7987
400/400 ━━━━━━━━━━━━━━━━━━━━ 142s 313ms/step - accuracy: 0.7780 - loss: 0.4067 - val_accuracy: 0.7987 - val_loss: 0.4005 - val_f1: 0.7987
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 96ms/step
 - val_f1: 0.8063
400/400 ━━━━━━━━━━━━━━━━━━━━ 138s 304ms/step - accuracy: 0.8057 - loss: 0.3854 - val_accuracy: 0.8169 - val_loss: 0.3807 - val_f1: 0.8063
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 85ms/step
 - val_f1: 0.7890
400/400 ━━━━━━━━━━━━━━━━━━━━ 143s 305ms/step - accuracy: 0.8305 - loss: 0.3512 - val_accuracy: 0.7775 - val_loss: 0.4114 - val_f1: 0.7890
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step
 - val_f1: 0.8111
400/400 ━━━━━━━━━━━━━━━━━━━━ 125s 312ms/step - accuracy: 0.8422 - loss: 0.3336 - val_accuracy: 0.8062 

## Teenager / Non-Teenager


In [ ]:
teenager_df['teenager_class'] = teenager_df['class'].apply(lambda x: 'teenager' if x == 'teenagers' else 'nonteenager')

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
teenager_df["label"] = label_encoder.fit_transform(teenager_df["teenager_class"])
num_classes = len(label_encoder.classes_)

### Vectorization/Embedding

In [ ]:
# Initialize tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Padding sequences to have the same length
max_len = 200  # Max length for padding
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Vocabulary size
vocab_size = len(word_index) + 1

In [ ]:
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

In [ ]:
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 191ms/step
 - val_f1: 0.7761
400/400 ━━━━━━━━━━━━━━━━━━━━ 246s 604ms/step - accuracy: 0.7602 - loss: 0.4726 - val_accuracy: 0.7922 - val_loss: 0.4083 - val_f1: 0.7761
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 175ms/step
 - val_f1: 0.7686
400/400 ━━━━━━━━━━━━━━━━━━━━ 259s 595ms/step - accuracy: 0.7960 - loss: 0.4024 - val_accuracy: 0.7891 - val_loss: 0.3990 - val_f1: 0.7686
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 189ms/step
 - val_f1: 0.7953
400/400 ━━━━━━━━━━━━━━━━━━━━ 260s 592ms/step - accuracy: 0.8080 - loss: 0.3873 - val_accuracy: 0.8034 - val_loss: 0.3856 - val_f1: 0.7953
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 18s 176ms/step
 - val_f1: 0.7965
400/400 ━━━━━━━━━━━━━━━━━━━━ 261s 591ms/step - accuracy: 0.8294 - loss: 0.3528 - val_accuracy: 0.8037 - val_loss: 0.3925 - val_f1: 0.7965
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 24s 238ms/step
 - val_f1: 0.8147
400/400 ━━━━━━━━━━━━━━━━━━━━ 266s 601ms/step - accuracy: 0.8301 - loss: 0.3487 - val_accuracy:

In [ ]:
print("Training Bi-LSTM with FastText Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 18s 177ms/step
 - val_f1: 0.7654
400/400 ━━━━━━━━━━━━━━━━━━━━ 242s 596ms/step - accuracy: 0.7525 - loss: 0.4873 - val_accuracy: 0.7722 - val_loss: 0.4323 - val_f1: 0.7654
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 18s 177ms/step
 - val_f1: 0.7682
400/400 ━━━━━━━━━━━━━━━━━━━━ 242s 605ms/step - accuracy: 0.7839 - loss: 0.4079 - val_accuracy: 0.7750 - val_loss: 0.4535 - val_f1: 0.7682
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 19s 191ms/step
 - val_f1: 0.7843
400/400 ━━━━━━━━━━━━━━━━━━━━ 278s 646ms/step - accuracy: 0.7914 - loss: 0.4023 - val_accuracy: 0.8016 - val_loss: 0.4325 - val_f1: 0.7843
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 18s 176ms/step
 - val_f1: 0.7626
400/400 ━━━━━━━━━━━━━━━━━━━━ 245s 604ms/step - accuracy: 0.8218 - loss: 0.3600 - val_accuracy: 0.7866 - val_loss: 0.4055 - val_f1: 0.7626
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 173ms/step
 - val_f1: 0.8025
400/400 ━━━━━━━━━━━━━━━━━━━━ 260s 599ms/step - accuracy: 0.8514 - loss: 0.3278 - val_accuracy:

In [ ]:
print("Training CNN with Word2Vec Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.7940
400/400 ━━━━━━━━━━━━━━━━━━━━ 25s 57ms/step - accuracy: 0.7352 - loss: 0.5898 - val_accuracy: 0.7912 - val_loss: 0.4059 - val_f1: 0.7940
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step
 - val_f1: 0.8070
400/400 ━━━━━━━━━━━━━━━━━━━━ 41s 57ms/step - accuracy: 0.8074 - loss: 0.3904 - val_accuracy: 0.8103 - val_loss: 0.3935 - val_f1: 0.8070
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.8138
400/400 ━━━━━━━━━━━━━━━━━━━━ 42s 58ms/step - accuracy: 0.8265 - loss: 0.3597 - val_accuracy: 0.8234 - val_loss: 0.3824 - val_f1: 0.8138
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.8082
400/400 ━━━━━━━━━━━━━━━━━━━━ 25s 62ms/step - accuracy: 0.8378 - loss: 0.3327 - val_accuracy: 0.8181 - val_loss: 0.3865 - val_f1: 0.8082
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.8040
400/400 ━━━━━━━━━━━━━━━━━━━━ 40s 59ms/step - accuracy: 0.8466 - loss: 0.3129 - val_accuracy: 0.8194 - val_loss: 

In [ ]:
print("Training CNN with FastText Embeddings...")
cnn_model = build_cnn_model(vocab_size, embedding_matrix_ft, num_classes)
train_and_evaluate(cnn_model, train_padded, train_labels, test_padded, test_labels)

Training CNN with FastText Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.6408
400/400 ━━━━━━━━━━━━━━━━━━━━ 25s 57ms/step - accuracy: 0.7052 - loss: 0.8155 - val_accuracy: 0.7484 - val_loss: 0.4657 - val_f1: 0.6408
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
 - val_f1: 0.6408
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 54ms/step - accuracy: 0.7511 - loss: 0.4715 - val_accuracy: 0.7484 - val_loss: 0.4603 - val_f1: 0.6408
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step
 - val_f1: 0.7165
400/400 ━━━━━━━━━━━━━━━━━━━━ 43s 60ms/step - accuracy: 0.7545 - loss: 0.4477 - val_accuracy: 0.7600 - val_loss: 0.4434 - val_f1: 0.7165
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
 - val_f1: 0.7633
400/400 ━━━━━━━━━━━━━━━━━━━━ 41s 59ms/step - accuracy: 0.7592 - loss: 0.4374 - val_accuracy: 0.7641 - val_loss: 0.4333 - val_f1: 0.7633
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step
 - val_f1: 0.7352
400/400 ━━━━━━━━━━━━━━━━━━━━ 40s 57ms/step - accuracy: 0.7681 - loss: 0.4187 - val_accuracy: 0.7653 - val_loss: 

In [ ]:
print("Training RNN with Word2Vec Embeddings...")
rnn_model = build_rnn_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(rnn_model, train_padded, train_labels, test_padded, test_labels)

Training RNN with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 107ms/step
 - val_f1: 0.6494
400/400 ━━━━━━━━━━━━━━━━━━━━ 138s 339ms/step - accuracy: 0.7350 - loss: 0.5581 - val_accuracy: 0.7509 - val_loss: 0.5191 - val_f1: 0.6494
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step
 - val_f1: 0.7195
400/400 ━━━━━━━━━━━━━━━━━━━━ 136s 339ms/step - accuracy: 0.7595 - loss: 0.5219 - val_accuracy: 0.7525 - val_loss: 0.4787 - val_f1: 0.7195
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 101ms/step
 - val_f1: 0.7231
400/400 ━━━━━━━━━━━━━━━━━━━━ 141s 338ms/step - accuracy: 0.7638 - loss: 0.4963 - val_accuracy: 0.7681 - val_loss: 0.4679 - val_f1: 0.7231
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 95ms/step
 - val_f1: 0.8087
400/400 ━━━━━━━━━━━━━━━━━━━━ 141s 336ms/step - accuracy: 0.7888 - loss: 0.4311 - val_accuracy: 0.8087 - val_loss: 0.3945 - val_f1: 0.8087
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step
 - val_f1: 0.8129
400/400 ━━━━━━━━━━━━━━━━━━━━ 153s 364ms/step - accuracy: 0.8177 - loss: 0.3733 - val_accuracy: 

### Tokenized

In [ ]:
label_encoder = LabelEncoder()
processed_df["label"] = label_encoder.fit_transform(processed_df["teenager_class"])
num_classes = len(label_encoder.classes_)

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(depression_df["tokens"], depression_df["label"], test_size=0.2, random_state=64)

# Train Word2Vec and FastText models
word2vec_model = gensim.models.Word2Vec(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)
fasttext_model = gensim.models.FastText(sentences=train_texts.tolist(), vector_size=100, window=5, min_count=1, workers=4)

# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts.apply(' '.join))  # Join tokens back to text for the tokenizer
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Create embedding matrices
embedding_matrix_w2v = np.zeros((vocab_size, 100))
embedding_matrix_ft = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix_w2v[i] = word2vec_model.wv[word]
    if word in fasttext_model.wv:
        embedding_matrix_ft[i] = fasttext_model.wv[word]

max_len = 100  # Max length for padding

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts.apply(' '.join))  # Join tokens for sequences
test_sequences = tokenizer.texts_to_sequences(test_texts.apply(' '.join))

# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post')

# Convert labels to categorical
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Train Bi-LSTM with Word2Vec embeddings
print("Training Bi-LSTM with Word2Vec Embeddings...")
bilstm_model = build_bilstm_model(vocab_size, embedding_matrix_w2v, num_classes)
train_and_evaluate(bilstm_model, train_padded, train_labels, test_padded, test_labels)

Training Bi-LSTM with Word2Vec Embeddings...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 83ms/step
 - val_f1: 0.7885
400/400 ━━━━━━━━━━━━━━━━━━━━ 128s 309ms/step - accuracy: 0.7518 - loss: 0.4708 - val_accuracy: 0.7841 - val_loss: 0.4099 - val_f1: 0.7885
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step
 - val_f1: 0.7844
400/400 ━━━━━━━━━━━━━━━━━━━━ 121s 303ms/step - accuracy: 0.7957 - loss: 0.3935 - val_accuracy: 0.7741 - val_loss: 0.4236 - val_f1: 0.7844
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 97ms/step
 - val_f1: 0.7760
400/400 ━━━━━━━━━━━━━━━━━━━━ 143s 305ms/step - accuracy: 0.8237 - loss: 0.3580 - val_accuracy: 0.8019 - val_loss: 0.4104 - val_f1: 0.7760
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step
 - val_f1: 0.8206
400/400 ━━━━━━━━━━━━━━━━━━━━ 143s 308ms/step - accuracy: 0.8336 - loss: 0.3517 - val_accuracy: 0.8231 - val_loss: 0.3861 - val_f1: 0.8206
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step
 - val_f1: 0.8075
400/400 ━━━━━━━━━━━━━━━━━━━━ 121s 302ms/step - accuracy: 0.8519 - loss: 0.3103 - val_accuracy: 0.821